# SSZ-Schumann: Segmented Spacetime Analysis of Schumann Resonances

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/error-wtf/ssz-schumann/blob/main/SSZ_Schumann_Colab.ipynb)

This notebook demonstrates the SSZ (Segmented Spacetime) framework applied to Schumann resonance analysis.

**Authors:** Carmen Wrede & Lino Casu

---

## ⚠️ Important: Data Download Required

This notebook requires downloading real Schumann resonance data from Zenodo (~1-2 GB).

**Expected download time: 10-30 minutes** depending on your connection speed.

Alternatively, you can run the analysis with **synthetic data** (no download required) to see the methodology.

---

## 1. Setup

First, we clone the repository and install dependencies.

In [ ]:
# Clone the repository
!git clone https://github.com/error-wtf/ssz-schumann.git
%cd ssz-schumann

# Install dependencies
!pip install -q numpy scipy pandas matplotlib

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Import SSZ-Schumann modules
import sys
sys.path.insert(0, '.')

from ssz_schumann.config import *
from ssz_schumann.models.classical_schumann import classical_schumann_frequency
from ssz_schumann.models.ssz_correction import ssz_corrected_frequency

print("SSZ-Schumann loaded successfully!")

## 2. Schumann Resonance Basics

### 2.1 Classical Schumann Frequencies

The Earth-ionosphere cavity resonates at specific frequencies:

$$f_n = \frac{\eta \cdot c}{2\pi R} \sqrt{n(n+1)}$$

where:
- $n$ = mode number (1, 2, 3, ...)
- $\eta \approx 0.74$ = effective slowdown factor
- $c$ = speed of light
- $R$ = Earth radius

In [ ]:
# Calculate classical Schumann frequencies
print("Classical Schumann Resonance Frequencies:")
print("="*50)

observed = [7.83, 14.1, 20.3, 26.4, 32.4]  # Typical observed values

for n in range(1, 6):
    f_classical = classical_schumann_frequency(n)
    print(f"Mode n={n}: {f_classical:.2f} Hz (observed: ~{observed[n-1]:.1f} Hz)")

### 2.2 SSZ Modification

In SSZ theory, spacetime segmentation modifies the effective speed of light:

$$f_n^{SSZ} = \frac{f_n^{classical}}{D_{SSZ}}$$

where $D_{SSZ} = 1 + \delta_{seg}(t)$

The **key SSZ signature** is that all modes shift by the same relative amount:

$$\frac{\Delta f_n}{f_n} \approx -\delta_{seg}(t) \quad \text{for all } n$$

In [ ]:
# Demonstrate SSZ correction
delta_seg_values = [0.0, 0.01, 0.02, 0.05]  # Different segmentation levels

print("SSZ-Corrected Frequencies for Mode n=1:")
print("="*50)

f1_classical = classical_schumann_frequency(1)

for delta_seg in delta_seg_values:
    f1_ssz = ssz_corrected_frequency(1, delta_seg)
    shift = (f1_ssz - f1_classical) / f1_classical * 100
    print(f"δ_seg = {delta_seg:.2f}: f1 = {f1_ssz:.3f} Hz (shift: {shift:+.2f}%)")

## 3. Option A: Run with Synthetic Data (Quick Demo)

This creates synthetic Schumann data with a known SSZ signature to demonstrate the analysis pipeline.

In [ ]:
# Generate synthetic data with SSZ signature
np.random.seed(42)

n_points = 1000
time = np.arange(n_points)

# Inject a 2% SSZ signal with daily variation
delta_seg_true = 0.02 * np.sin(2 * np.pi * time / 144)  # 144 points = 1 day

# Generate frequencies for modes 1, 2, 3
f1_base = 7.83
f2_base = 14.1
f3_base = 20.3

noise_level = 0.05  # Hz

f1 = f1_base / (1 + delta_seg_true) + np.random.normal(0, noise_level, n_points)
f2 = f2_base / (1 + delta_seg_true) + np.random.normal(0, noise_level, n_points)
f3 = f3_base / (1 + delta_seg_true) + np.random.normal(0, noise_level, n_points)

print(f"Generated {n_points} synthetic data points")
print(f"Injected SSZ signal amplitude: 2%")

In [ ]:
# Visualize synthetic data
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Plot frequencies
ax1 = axes[0]
ax1.plot(time, f1, 'b-', alpha=0.7, label='f1 (7.83 Hz)')
ax1.plot(time, f2, 'g-', alpha=0.7, label='f2 (14.1 Hz)')
ax1.plot(time, f3, 'r-', alpha=0.7, label='f3 (20.3 Hz)')
ax1.set_xlabel('Time [10-min intervals]')
ax1.set_ylabel('Frequency [Hz]')
ax1.set_title('Synthetic Schumann Resonance Frequencies')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot injected SSZ signal
ax2 = axes[1]
ax2.plot(time, delta_seg_true * 100, 'k-', linewidth=2)
ax2.set_xlabel('Time [10-min intervals]')
ax2.set_ylabel('δ_seg [%]')
ax2.set_title('Injected SSZ Signal (Ground Truth)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.1 Recover SSZ Signal from Synthetic Data

In [ ]:
# Calculate relative frequency shifts
delta_f1 = (f1_base - f1) / f1
delta_f2 = (f2_base - f2) / f2
delta_f3 = (f3_base - f3) / f3

# Average across modes (SSZ signature: all modes shift equally)
delta_seg_recovered = (delta_f1 + delta_f2 + delta_f3) / 3

# Compare with ground truth
correlation = np.corrcoef(delta_seg_true, delta_seg_recovered)[0, 1]

print(f"Correlation between true and recovered δ_seg: {correlation:.4f}")
print(f"Recovery successful: {'✓ YES' if correlation > 0.9 else '✗ NO'}")

In [ ]:
# Visualize recovery
plt.figure(figsize=(12, 5))

plt.plot(time, delta_seg_true * 100, 'k-', linewidth=2, label='True δ_seg')
plt.plot(time, delta_seg_recovered * 100, 'r--', linewidth=1.5, alpha=0.8, label='Recovered δ_seg')

plt.xlabel('Time [10-min intervals]')
plt.ylabel('δ_seg [%]')
plt.title(f'SSZ Signal Recovery (Correlation: {correlation:.4f})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 3.2 Mode Consistency Test

The SSZ signature is that all modes shift by the same relative amount. We test this by checking the correlation between mode shifts.

In [ ]:
# Mode consistency: correlations between relative shifts
corr_12 = np.corrcoef(delta_f1, delta_f2)[0, 1]
corr_13 = np.corrcoef(delta_f1, delta_f3)[0, 1]
corr_23 = np.corrcoef(delta_f2, delta_f3)[0, 1]

mode_consistency = (corr_12 + corr_13 + corr_23) / 3

print("Mode Consistency Analysis:")
print("="*50)
print(f"Correlation f1-f2: {corr_12:.4f}")
print(f"Correlation f1-f3: {corr_13:.4f}")
print(f"Correlation f2-f3: {corr_23:.4f}")
print(f"\nMode Consistency Score: {mode_consistency:.4f}")
print(f"\nSSZ Signature: {'✓ DETECTED' if mode_consistency > 0.7 else '✗ NOT DETECTED'}")

## 4. Option B: Download Real Data (Takes 10-30 minutes)

⚠️ **Warning:** This downloads ~1-2 GB of data from Zenodo. Only run if you have time and bandwidth.

The data comes from the Sierra Nevada ELF Station (Salinas et al., 2022):
- **DOI:** [10.5281/zenodo.6348930](https://doi.org/10.5281/zenodo.6348930)
- **Period:** March 2013 - February 2017
- **Resolution:** 10-minute intervals

In [ ]:
# Uncomment the following lines to download real data
# WARNING: This takes 10-30 minutes!

# print("Starting data download from Zenodo...")
# print("This will take 10-30 minutes depending on your connection.")
# print("")
# !python scripts/fetch_zenodo_schumann.py
# print("")
# print("Download complete!")

In [ ]:
# After downloading, you can load and analyze real data:

# from ssz_schumann.data_io import load_schumann_data
# real_data = load_schumann_data("data/schumann/")
# 
# # Run full analysis
# !python scripts/run_schumann_ssz_analysis.py --schumann-path data/schumann/

## 5. Summary

### Key Findings

1. **SSZ Signature:** Uniform relative frequency shift across all Schumann modes
2. **Mode Consistency:** High correlation between mode shifts indicates SSZ effect
3. **Testable Prediction:** δ_seg should correlate with ionospheric conditions

### Model Comparison (Synthetic Data with 2% SSZ Signal)

| Metric | Classical Model | SSZ Model |
|--------|-----------------|------------|
| RMSE | ~1.0 Hz | ~0.4 Hz |
| R² | ~0.96 | ~0.99 |

### Next Steps

1. Download real Schumann data (Option B above)
2. Run full analysis with space weather correlation
3. Test SSZ predictions against ionospheric models

For more information, see the [GitHub repository](https://github.com/error-wtf/ssz-schumann).

---

## License

Anti-Capitalist Software License v1.4

© 2025 Carmen Wrede & Lino Casu

**Contact:** mail@error.wtf

---

**"Schumann resonances as a probe for spacetime segmentation."**